In [1]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg
from std_msgs.msg import String
from moveit_commander.conversions import pose_to_list

In [2]:
import cv2 
import tf as tf
import tf2_ros as tf2
import rospy
import numpy as np
import ros_numpy
from std_msgs.msg import String
from tmc_msgs.msg import Voice
from geometry_msgs.msg import Twist, WrenchStamped, TransformStamped
from sensor_msgs.msg import Image as ImageMsg, PointCloud2
import tmc_control_msgs.msg
import trajectory_msgs.msg
class TF_MANAGER():
    def __init__(self):
        self._tfbuff = tf2.Buffer()
        self._lis = tf2.TransformListener(self._tfbuff)
        self._tf_static_broad = tf2.StaticTransformBroadcaster()
        self._broad = tf2.TransformBroadcaster()

    def _fillMsg(self, pos = [0,0,0], rot = [0,0,0,1] ,point_name ='', ref="map"):
        TS = TransformStamped()
        TS.header.stamp = rospy.Time.now()
        TS.header.frame_id = ref
        TS.child_frame_id = point_name
        TS.transform.translation.x = pos[0]
        TS.transform.translation.y = pos[1]
        TS.transform.translation.z = pos[2]
        TS.transform.rotation.x = rot[0]
        TS.transform.rotation.y = rot[1]
        TS.transform.rotation.z = rot[2]
        TS.transform.rotation.w = rot[3]
        return TS

    def pub_tf(self, pos = [0,0,0], rot = [0,0,0,1] ,point_name ='', ref="map"):
        dinamic_ts = self._fillMsg(pos, rot, point_name, ref)
        self._broad.sendTransform(dinamic_ts)

    def pub_static_tf(self, pos = [0,0,0], rot = [0,0,0,1] ,point_name ='', ref="map"):
        static_ts = self._fillMsg(pos, rot, point_name, ref)
        self._tf_static_broad.sendTransform(static_ts)

    def change_ref_frame_tf(self, point_name = '', new_frame = 'map'):
        try:
            traf = self._tfbuff.lookup_transform(new_frame, point_name, rospy.Time(0))
            translation, rotational = self.tf2_obj_2_arr(traf)
            self.pub_static_tf(pos = translation, rot = rotational, point_name = point_name, ref = new_frame)
            return True
        except:
            return False

    def getTF(self, target_frame='', ref_frame='map'):
        try:
            tf = self._tfbuff.lookup_transform(ref_frame, target_frame, rospy.Time(0))
            return self.tf2_obj_2_arr(tf)
        except:
            return [False,False]

    def tf2_obj_2_arr(self, transf):
        pos = []
        pos.append(transf.transform.translation.x)
        pos.append(transf.transform.translation.y)
        pos.append(transf.transform.translation.z)
    
        rot = []
        rot.append(transf.transform.rotation.x)
        rot.append(transf.transform.rotation.y)
        rot.append(transf.transform.rotation.z)
        rot.append(transf.transform.rotation.w)

        return [pos, rot]
class GRIPPER():
    def __init__(self):
        self._grip_cmd_pub = rospy.Publisher('/hsrb/gripper_controller/command',
                               trajectory_msgs.msg.JointTrajectory, queue_size=100)
        self._grip_cmd_force = rospy.Publisher('/hsrb/gripper_controller/grasp/goal',
        			tmc_control_msgs.msg.GripperApplyEffortActionGoal, queue_size=100)
        			
        self._joint_name = "hand_motor_joint"
        self._position = 0.5
        self._velocity = 0.5
        self._effort = 0.0
        self._duration = 1

    def _manipulate_gripper(self):
        traj = trajectory_msgs.msg.JointTrajectory()
        traj.joint_names = [self._joint_name]
        p = trajectory_msgs.msg.JointTrajectoryPoint()
        p.positions = [self._position]
        p.velocities = [self._velocity]
        p.accelerations = []
        p.effort = [self._effort]
        p.time_from_start = rospy.Duration(self._duration)
        traj.points = [p]
        self._grip_cmd_pub.publish(traj)
        
    def _apply_force(self):
        app_force = tmc_control_msgs.msg.GripperApplyEffortActionGoal()
        app_force.goal.effort = -0.5
        self._grip_cmd_force.publish(app_force)
        
    def change_velocity(self, newVel):
        self._velocity = newVel
    
    def open(self):
        self._position = 1.23
        self._effort = 0
        self._manipulate_gripper()

    def steady(self):
        self._position = -0.82
        self._effort = -0.3
        self._manipulate_gripper()
        
    def close(self):
        self._position = -0.82
        self._effort = -0.3
        self._manipulate_gripper()
        self._apply_force()
        rospy.sleep(0.8)

In [3]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node("Tutorial", anonymous=True)
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()


[ WARN] [1670431286.294305763]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1670431286.296984521]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1670431286.297016101]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1670431286.297647891]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1670431286.297669045]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1670431311.605128489]: Kinematics solver doesn't support #attempts any

In [4]:
display_trajectory_publisher = rospy.Publisher(
    "/move_group/display_planned_path",
    moveit_msgs.msg.DisplayTrajectory,
    queue_size=20,
)

[ WARN] [1667835892.861681281, 2559.973000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2560.011000 according to authority unknown_publisher


In [5]:
# We can get a list of all the groups in the robot:
group_names = robot.get_group_names()
print(">>>Available Planning Groups:", robot.get_group_names())

# Sometimes for debugging it is useful to print the entire state of the
# robot:
print(">>>Printing robot state")
print(robot.get_current_state())


>>>Available Planning Groups: ['arm', 'base', 'gripper', 'head', 'whole_body', 'whole_body_light', 'whole_body_weighted']
>>>Printing robot state
joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "odom"
  name: 
    - arm_lift_joint
    - arm_flex_joint
    - arm_roll_joint
    - wrist_flex_joint
    - wrist_roll_joint
    - wrist_ft_sensor_frame_joint
    - hand_l_proximal_joint
    - hand_l_spring_proximal_joint
    - hand_l_mimic_distal_joint
    - hand_l_distal_joint
    - hand_motor_joint
    - hand_r_proximal_joint
    - hand_r_spring_proximal_joint
    - hand_r_mimic_distal_joint
    - hand_r_distal_joint
    - base_b_bumper_joint
    - base_f_bumper_joint
    - base_roll_joint
    - base_l_drive_wheel_joint
    - base_l_passive_wheel_x_frame_joint
    - base_l_passive_wheel_y_frame_joint
    - base_l_passive_wheel_z_joint
    - base_r_drive_wheel_joint
    - base_r_passive_wheel_x_frame_joint
    - base_r_passive_wheel_y_frame_jo

[ WARN] [1667835900.284779224, 2561.399000000]: Joint values for monitored state are requested but the full state is not known


In [4]:
head = moveit_commander.MoveGroupCommander('head')
whole_body = moveit_commander.MoveGroupCommander('whole_body_light')
# wb = moveit_commander.MoveGroupCommander('whole_body')
# wbw = moveit_commander.MoveGroupCommander('whole_body_weighted')
arm =  moveit_commander.MoveGroupCommander('arm')
#gripper =  moveit_commander.MoveGroupCommander('gripper')

[ WARN] [1670263604.868209445, 2353.901000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2353.933000 according to authority unknown_publisher
[ WARN] [1670263618.434195146, 2355.655000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2355.683000 according to authority unknown_publisher


In [5]:
arm.get_current_joint_values()


[-3.4570746174830528e-06,
 -0.00010566412765911082,
 9.799561571099957e-06,
 -0.00011270030975740752,
 -3.19085490332327e-05,
 0.0]

In [8]:
# We can get the name of the reference frame for this robot:
arm_planning_frame = arm.get_planning_frame()
head_planning_frame = head.get_planning_frame()
wb_planning_frame = whole_body.get_planning_frame()

print(f">>>Planning frame: {arm_planning_frame}, {head_planning_frame}, {wb_planning_frame}")

# We can also print the name of the end-effector link for this group:
eef_link = arm.get_end_effector_link()
print(f">>>End effector link: {eef_link}")

>>>Planning frame: odom, odom, odom
>>>End effector link: hand_palm_link


[ WARN] [1667836697.903117480, 2750.419000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2750.472000 according to authority unknown_publisher
[ WARN] [1667836699.243316981, 2750.688000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2750.722000 according to authority unknown_publisher
[ WARN] [1667836728.476342306, 2757.544000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2757.606000 according to authority unknown_publisher
[ WARN] [1667836803.668930858, 2775.087000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2775.111000 according to authority unknown_publisher
[ WARN] [1667836810.312684831, 2776.498000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2776.544000 according to authority unknown_publisher


In [6]:
# We get the joint values from the group and change some of the values:
joint_goal = arm.get_current_joint_values()
joint_goal[0] = 0.0
joint_goal[1] = 0.0
joint_goal[2] = 0.0
joint_goal[3] = -1.57
joint_goal[4] = 0.0
joint_goal[5] = 0.0
arm.set_joint_value_target(joint_goal)
arm.go()

[ WARN] [1670263662.324063587, 2361.686000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2361.730000 according to authority unknown_publisher
[ WARN] [1670263683.455863144, 2364.463000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2364.482000 according to authority unknown_publisher


True

In [7]:
tf_man = TF_MANAGER()

In [8]:
# rospy.sleep(0.5)
_, rot = tf_man.getTF(target_frame='hand_palm_link', ref_frame='odom')
tf_man.pub_static_tf(pos=[0.4,0.1,0.8], rot = rot, point_name='goal', ref='odom')
trans, rot = tf_man.getTF(target_frame='goal', ref_frame='odom')

print(trans)
pose_goal = geometry_msgs.msg.Pose()
pose_goal.orientation.w = rot[3]
pose_goal.orientation.x = rot[0]
pose_goal.orientation.y = rot[1]
pose_goal.orientation.z = rot[2]
# pose_goal.orientation.
pose_goal.position.x = trans[0]
pose_goal.position.y = trans[1]
pose_goal.position.z = trans[2]

whole_body.set_start_state_to_current_state()
whole_body.set_pose_target(pose_goal)

[0.4, 0.1, 0.8]


[ WARN] [1670263730.973660119, 2371.827000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2371.854000 according to authority /pose_integrator
[ WARN] [1670263730.976281880, 2371.827000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2371.854000 according to authority unknown_publisher


In [9]:
# `go()` returns a boolean indicating whether the planning and execution was successful.
whole_body.go(wait=True)
# Calling `stop()` ensures that there is no residual movement
#whole_body.stop()
# It is always good to clear your targets after planning with poses.
# Note: there is no equivalent function for clear_joint_value_targets().
whole_body.clear_pose_targets()

[ WARN] [1670263752.736890882, 2374.812000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2374.833000 according to authority /pose_integrator
[ WARN] [1670263752.737059904, 2374.812000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2374.833000 according to authority unknown_publisher
[ WARN] [1670263757.043198147, 2375.393000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2375.432000 according to authority /pose_integrator
[ WARN] [1670263757.049156578, 2375.393000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2375.432000 according to authority unknown_publisher
[ WARN] [1670263757.352991266, 2375.497000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2375.567000 according to authority unknown_publisher
[ WARN] [1670263757.353119872, 2375.497000000]: TF_REPEATED_DATA ignoring data with redundant timestam

[ WARN] [1670264220.304634542, 2444.491000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2444.530000 according to authority unknown_publisher
[ WARN] [1670264220.309741160, 2444.491000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2444.530000 according to authority /pose_integrator
[ WARN] [1670264242.397133276, 2448.512000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2448.512000 according to authority unknown_publisher
[ WARN] [1670264242.403290404, 2448.512000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2448.512000 according to authority /pose_integrator


In [13]:
arm.set_named_target('neutral')
arm.go()


[ WARN] [1667837189.069014609, 2860.606000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2860.638000 according to authority /pose_integrator
[ WARN] [1667837189.069355584, 2860.606000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2860.638000 according to authority unknown_publisher
[ WARN] [1667837206.786694750, 2864.330000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2864.348000 according to authority /pose_integrator
[ WARN] [1667837206.788579540, 2864.330000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2864.348000 according to authority unknown_publisher
[ WARN] [1667837226.062723227, 2868.661000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2868.752000 according to authority /pose_integrator
[ WARN] [1667837226.062881704, 2868.661000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

True

[ WARN] [1667837256.760109281, 2875.419000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2875.454000 according to authority /pose_integrator
[ WARN] [1667837256.763198630, 2875.419000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2875.454000 according to authority unknown_publisher
[ WARN] [1667837300.081208869, 2884.307000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2884.353000 according to authority /pose_integrator
[ WARN] [1667837300.081766553, 2884.307000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2884.353000 according to authority unknown_publisher
[ WARN] [1667837307.347098638, 2885.736000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2885.785000 according to authority /pose_integrator
[ WARN] [1667837307.347384168, 2885.736000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

In [10]:
# Cartesian Paths
waypoints = []
scale = 1

wpose = whole_body.get_current_pose().pose
wpose.position.z -= scale * 0.1  # First move up (z)
wpose.position.y += scale * 0.2  # and sideways (y)
waypoints.append(copy.deepcopy(wpose))

wpose.position.x += scale * 0.1  # Second move forward/backwards in (x)
waypoints.append(copy.deepcopy(wpose))

wpose.position.y -= scale * 0.1  # Third move sideways (y)
waypoints.append(copy.deepcopy(wpose))

# We want the Cartesian path to be interpolated at a resolution of 1 cm
# which is why we will specify 0.01 as the eef_step in Cartesian
# translation.  We will disable the jump threshold by setting it to 0.0,
# ignoring the check for infeasible jumps in joint space, which is sufficient
# for this tutorial.
(plan, fraction) = whole_body.compute_cartesian_path(
    waypoints, 0.01, 0.0  # waypoints to follow  # eef_step
)  # jump_threshold

# Note: We are just planning, not asking move_group to actually move the robot yet:
print(f'fraction : {fraction*100}%')

fraction : 100.0%


In [11]:
##Display trajectory
display_trajectory = moveit_msgs.msg.DisplayTrajectory()
display_trajectory.trajectory_start = robot.get_current_state()
display_trajectory.trajectory.append(plan)
# Publish
display_trajectory_publisher.publish(display_trajectory)

NameError: name 'display_trajectory_publisher' is not defined

[ WARN] [1670264283.113627825, 2454.370000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2454.395000 according to authority unknown_publisher
[ WARN] [1670264283.115522512, 2454.370000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2454.395000 according to authority /pose_integrator


In [12]:
##Execute plan
whole_body.execute(plan, wait=True)

[ WARN] [1670264311.636685848, 2458.113000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2458.131000 according to authority /pose_integrator
[ WARN] [1670264311.638084823, 2458.113000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2458.131000 according to authority unknown_publisher
[ WARN] [1670264322.280984324, 2459.941000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2459.968000 according to authority /pose_integrator
[ WARN] [1670264322.281999155, 2459.941000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2459.968000 according to authority unknown_publisher
[ WARN] [1670264340.539852869, 2462.674000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2462.714000 according to authority /pose_integrator
[ WARN] [1670264340.540138298, 2462.674000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

True

[ WARN] [1670264383.122084868, 2469.240000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2469.267000 according to authority /pose_integrator
[ WARN] [1670264383.122382432, 2469.240000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2469.267000 according to authority unknown_publisher


In [17]:
##Adding objects to planning scene
box_pose = geometry_msgs.msg.PoseStamped()
box_pose.header.frame_id = "hand_palm_link"
box_pose.pose.orientation.w = 1.0
box_pose.pose.position.z =  0.20  # below the panda_hand frame
box_name = "box"
scene.add_box(box_name, box_pose, size=(0.075, 0.05, 0.075))

[ WARN] [1667837502.279902097, 2925.907000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2925.962000 according to authority /pose_integrator
[ WARN] [1667837502.280575207, 2925.907000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2925.962000 according to authority unknown_publisher
[ WARN] [1667837507.058339141, 2926.974000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2926.974000 according to authority /pose_integrator
[ WARN] [1667837507.058671602, 2926.974000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2926.974000 according to authority unknown_publisher
[ WARN] [1667837522.278307745, 2930.089000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2930.148000 according to authority /pose_integrator
[ WARN] [1667837522.278472764, 2930.089000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

In [18]:
##attaching object to the gripper
grasping_group = "gripper"
touch_links = robot.get_link_names(group=grasping_group)
scene.attach_box(eef_link, box_name, touch_links=touch_links)

[ WARN] [1667837549.671811358, 2935.454000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2935.489000 according to authority unknown_publisher
[ WARN] [1667837549.674718782, 2935.454000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2935.489000 according to authority /pose_integrator
[ WARN] [1667837559.378000713, 2937.294000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2937.332000 according to authority unknown_publisher
[ WARN] [1667837559.380357403, 2937.294000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2937.332000 according to authority /pose_integrator
[ WARN] [1667837563.409624524, 2938.238000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2938.281000 according to authority /pose_integrator
[ WARN] [1667837563.412051428, 2938.238000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

In [20]:
##Detaching objects
# scene.remove_attached_object(eef_link, name=box_name)

##Remove objects
scene.remove_world_object(box_name)

In [21]:
grip = GRIPPER()

[ WARN] [1667837627.000788691, 2951.227000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2951.255000 according to authority /pose_integrator
[ WARN] [1667837627.004142522, 2951.227000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2951.255000 according to authority unknown_publisher
[ WARN] [1667837637.413453275, 2953.120000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2953.157000 according to authority /pose_integrator
[ WARN] [1667837637.415793597, 2953.120000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2953.157000 according to authority unknown_publisher


In [22]:
grip.open()

[ WARN] [1667837652.016656838, 2956.250000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2956.301000 according to authority /pose_integrator
[ WARN] [1667837652.016935439, 2956.250000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2956.301000 according to authority unknown_publisher
[ WARN] [1667837665.241479325, 2958.675000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.721000 according to authority /pose_integrator
[ WARN] [1667837665.242194364, 2958.675000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.721000 according to authority unknown_publisher
[ WARN] [1667837665.537795807, 2958.788000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.833000 according to authority /pose_integrator
[ WARN] [1667837665.538775294, 2958.788000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

In [ ]:
arm.set_named_target('neutral')
arm.go()

In [ ]:
whole_body.set_num_planning_attempts(10)
whole_body.set_planning_time(5)

In [ ]:
# Cartesian Paths
waypoints = []
scale = 1

wpose = whole_body.get_current_pose().pose
wpose.position.x += scale * 0.2  # First move up (z)
# wpose.position.y += scale * 0.2  # and sideways (y)
waypoints.append(copy.deepcopy(wpose))

# wpose.position.x += scale * 0.1  # Second move forward/backwards in (x)
# waypoints.append(copy.deepcopy(wpose))

# wpose.position.y -= scale * 0.1  # Third move sideways (y)
# waypoints.append(copy.deepcopy(wpose))

# We want the Cartesian path to be interpolated at a resolution of 1 cm
# which is why we will specify 0.01 as the eef_step in Cartesian
# translation.  We will disable the jump threshold by setting it to 0.0,
# ignoring the check for infeasible jumps in joint space, which is sufficient
# for this tutorial.
(plan, fraction) = whole_body.compute_cartesian_path(
    waypoints, 0.01, 0.0  # waypoints to follow  # eef_step
)  # jump_threshold

# Note: We are just planning, not asking move_group to actually move the robot yet:
print(f'fraction : {fraction*100}%')

In [13]:
# trans, rot  = tf_man.getTF(target_frame='hand_palm_link',ref_frame='odom')
tf_man.pub_static_tf(point_name='grasp', ref='hand_palm_link', pos=[0,0,0.1])

In [14]:
tf_man.change_ref_frame_tf(point_name='grasp',new_frame='odom')

True

[ WARN] [1670264397.452075797, 2471.210000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2471.237000 according to authority /pose_integrator
[ WARN] [1670264397.452215147, 2471.210000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2471.237000 according to authority unknown_publisher


In [15]:
trans, rot = tf_man.getTF(target_frame='grasp', ref_frame='odom')
pose_goal = geometry_msgs.msg.Pose()
pose_goal.orientation.w = rot[3]
pose_goal.orientation.x = rot[0]
pose_goal.orientation.y = rot[1]
pose_goal.orientation.z = rot[2]
# pose_goal.orientation.
pose_goal.position.x = trans[0]
pose_goal.position.y = trans[1]
pose_goal.position.z = trans[2]

whole_body.set_start_state_to_current_state()
whole_body.set_pose_target(pose_goal)
whole_body.go()

[ WARN] [1670264423.101147050, 2474.880000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2474.935000 according to authority unknown_publisher
[ WARN] [1670264423.101238376, 2474.880000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2474.935000 according to authority /pose_integrator
[ WARN] [1670264424.246895527, 2475.095000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2475.149000 according to authority /pose_integrator
[ WARN] [1670264424.250402517, 2475.095000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2475.149000 according to authority unknown_publisher
[ WARN] [1670264427.318557486, 2475.647000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2475.684000 according to authority /pose_integrator
[ WARN] [1670264427.319322648, 2475.647000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

True

[ WARN] [1670264464.443976364, 2480.471000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2480.511000 according to authority /pose_integrator
[ WARN] [1670264464.446710544, 2480.471000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2480.511000 according to authority unknown_publisher


In [27]:
grip.close()

[ WARN] [1667837953.705212932, 3018.051000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3018.081000 according to authority /pose_integrator
[ WARN] [1667837953.705381052, 3018.051000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3018.081000 according to authority unknown_publisher
[ WARN] [1667837976.185259636, 3022.617000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3022.675000 according to authority /pose_integrator
[ WARN] [1667837976.185439401, 3022.617000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3022.675000 according to authority unknown_publisher
[ WARN] [1667837977.950027824, 3023.070000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3023.112000 according to authority /pose_integrator
[ WARN] [1667837977.952755108, 3023.070000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

[ WARN] [1667838184.131470007, 3068.597000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3068.634000 according to authority /pose_integrator
[ WARN] [1667838184.131941451, 3068.597000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3068.634000 according to authority unknown_publisher
[ WARN] [1667838186.199684180, 3069.048000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3069.064000 according to authority /pose_integrator
[ WARN] [1667838186.200890296, 3069.048000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3069.064000 according to authority unknown_publisher
[ WARN] [1667838202.239494181, 3072.448000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3072.481000 according to authority /pose_integrator
[ WARN] [1667838202.241008935, 3072.448000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

[ WARN] [1667838474.051744683, 3138.003000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3138.026000 according to authority /pose_integrator
[ WARN] [1667838474.051894400, 3138.003000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3138.026000 according to authority unknown_publisher
[ WARN] [1667838500.350564181, 3145.052000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3145.052000 according to authority unknown_publisher
[ WARN] [1667838500.352426041, 3145.052000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3145.052000 according to authority /pose_integrator
[ WARN] [1667838502.347291394, 3145.581000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3145.650000 according to authority /pose_integrator
[ WARN] [1667838502.349961351, 3145.581000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

[ WARN] [1667838784.105269588, 3215.730000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3215.769000 according to authority /pose_integrator
[ WARN] [1667838784.109251135, 3215.730000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3215.769000 according to authority unknown_publisher
[ WARN] [1667838785.463593332, 3216.083000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3216.111000 according to authority /pose_integrator
[ WARN] [1667838785.464192647, 3216.083000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3216.111000 according to authority unknown_publisher
[ WARN] [1667838788.656141449, 3216.877000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3216.898000 according to authority /pose_integrator
[ WARN] [1667838788.656306765, 3216.877000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

[ WARN] [1667839047.788934365, 3276.596000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3276.634000 according to authority unknown_publisher
[ WARN] [1667839047.789085307, 3276.596000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3276.634000 according to authority /pose_integrator
[ WARN] [1667839051.267447092, 3277.224000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3277.284000 according to authority /pose_integrator
[ WARN] [1667839051.268409059, 3277.224000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3277.284000 according to authority unknown_publisher
[ WARN] [1667839063.120903174, 3279.182000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3279.217000 according to authority /pose_integrator
[ WARN] [1667839063.121515402, 3279.182000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

[ WARN] [1667839336.179543165, 3336.485000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3336.537000 according to authority /pose_integrator
[ WARN] [1667839336.180311208, 3336.485000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3336.537000 according to authority unknown_publisher
[ WARN] [1667839361.600646801, 3341.680000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3341.705000 according to authority /pose_integrator
[ WARN] [1667839361.603569607, 3341.680000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3341.705000 according to authority unknown_publisher


In [ ]:
arm.set_named_target('neutral')
arm.go()

[ WARN] [1670264520.207745480, 2488.672000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2488.704000 according to authority unknown_publisher
[ WARN] [1670264520.208097141, 2488.672000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2488.704000 according to authority /pose_integrator


In [ ]:
wb = moveit_commander.MoveGroupCommander('whole_body_weighted')

In [ ]:
trans, rot = tf_man.getTF(target_frame='goal', ref_frame='odom')
wb.set_position_target(trans)
wb.go()

In [ ]:
trans, _ = tf_man.getTF(target_frame='goal', ref_frame='odom')
_, rot = tf_man.getTF(target_frame='hand_palm_link', ref_frame='odom')
tf_man.pub_static_tf(pos=trans, rot=rot, point_name='goal', ref='odom',)